# Analyze

In [1]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from functools import reduce

import yfinance as yf

import pyarrow as pa
import pyarrow.parquet as pq

# Ignore DeprecationWarnings
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

# Enable multiple outputs from each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Make columns wider
pd.set_option('max_colwidth',1024*1024)

%matplotlib inline

In [2]:
DATADIR = "/Users/adarshrp/Projects/Trading"
dfG250 = pd.read_csv(DATADIR + "/MarketSmith/Extracts/Growth250.csv")
dfUniverse = pd.read_csv(DATADIR + "/MarketSmith/Extracts/MarketSmithUniverseCurated.csv")

In [3]:
def rename_columns(oldColumns):
    newColumns = oldColumns
    newColumns = [re.sub("/", "", col) for col in newColumns]
    newColumns = [re.sub("[\-\(\) ]", "_", col.replace("%", "Perc").replace("$", "Dollar")) for col in newColumns]
    newColumns = [re.sub("[_]+", "_", col) for col in newColumns]
    newColumns = [re.sub("[_]$", "", col) for col in newColumns]
    return newColumns

In [4]:
import re

universe = spark.read.option("header", True).option("inferSchema", True).csv(DATADIR + "/MarketSmith/Extracts/MarketSmithUniverseCurated.csv")
#universe.printSchema()

#oldColumns = universe.columns
#newColumns = rename_columns(oldColumns)

#universe = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), universe)
universe.printSchema()

universe.createOrReplaceTempView("universe")


root
 |-- Order: integer (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Industry_Name: string (nullable = true)
 |-- Ind_Group_Rank: integer (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Shares_1000s: integer (nullable = true)
 |-- Shares_in_Float_1000s: integer (nullable = true)
 |-- Market_Cap_mil: double (nullable = true)
 |-- Funds_Perc: double (nullable = true)
 |-- Funds_Perc_Increase: double (nullable = true)
 |-- Mgmt_Perc: double (nullable = true)
 |-- Number_of_Funds: double (nullable = true)
 |-- Inst_Ownership_Lss_Median: double (nullable = true)
 |-- Sponsor_Rating: string (nullable = true)
 |-- Enterprise_Val_mil: double (nullable = true)
 |-- EPS_Rating: double (nullable = true)
 |-- RS_Rating: double (nullable = true)
 |-- SMR_Rating: string (nullable = true)
 |-- AD_Rating: string (nullable = true)
 |-- Comp_Rating: double (nullable = true)
 |-- Ind_Group_RS: string (nullable = true)
 |-- EPS_Lst_Rptd: st

In [6]:
#parquetFile = spark.read.parquet("Data/ADBE-2001-01-01-2021-12-31.parquet")
prices = spark.read.parquet(DATADIR + "/Data/Enriched")

prices.createOrReplaceTempView("prices")



In [7]:
prices.printSchema()


root
 |-- index: long (nullable = true)
 |-- Ticker: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- SMA20: double (nullable = true)
 |-- SMA50: double (nullable = true)
 |-- SMA150: double (nullable = true)
 |-- SMA200: double (nullable = true)
 |-- HIGH52WK: double (nullable = true)
 |-- LOW52WK: double (nullable = true)
 |-- Volume14MA: double (nullable = true)
 |-- Strength21: double (nullable = true)
 |-- Strength42: double (nullable = true)
 |-- Strength63: double (nullable = true)
 |-- Strength: double (nullable = true)
 |-- PrevClose: double (nullable = true)
 |-- ATR: double (nullable = true)



In [12]:
spark.sql("""
  SELECT count(*) from prices
""").show(200)

+--------+
|count(1)|
+--------+
| 2891621|
+--------+



In [13]:
# Filter stocks based on following criteria:
# - Minervini trend template
# - Computers related
# - Earnings > 20, Sales > 15
# Ordered by 21-day relative strength
q = spark.sql("""
    SELECT Ticker, Date
    FROM prices P
    where (Ticker = 'ADBE' or Ticker = 'IBM')
""")
q.show(200)


#for ticker in q.toPandas().Ticker:
#    print(ticker)

+------+----------+
|Ticker|      Date|
+------+----------+
|  ADBE|2018-01-02|
|  ADBE|2018-01-03|
|  ADBE|2018-01-04|
|  ADBE|2018-01-05|
|  ADBE|2018-01-08|
|  ADBE|2018-01-09|
|  ADBE|2018-01-10|
|  ADBE|2018-01-11|
|  ADBE|2018-01-12|
|  ADBE|2018-01-16|
|  ADBE|2018-01-17|
|  ADBE|2018-01-18|
|  ADBE|2018-01-19|
|  ADBE|2018-01-22|
|  ADBE|2018-01-23|
|  ADBE|2018-01-24|
|  ADBE|2018-01-25|
|  ADBE|2018-01-26|
|  ADBE|2018-01-29|
|  ADBE|2018-01-30|
|  ADBE|2018-01-31|
|  ADBE|2018-02-01|
|  ADBE|2018-02-02|
|  ADBE|2018-02-05|
|  ADBE|2018-02-06|
|  ADBE|2018-02-07|
|  ADBE|2018-02-08|
|  ADBE|2018-02-09|
|  ADBE|2018-02-12|
|  ADBE|2018-02-13|
|  ADBE|2018-02-14|
|  ADBE|2018-02-15|
|  ADBE|2018-02-16|
|  ADBE|2018-02-20|
|  ADBE|2018-02-21|
|  ADBE|2018-02-22|
|  ADBE|2018-02-23|
|  ADBE|2018-02-26|
|  ADBE|2018-02-27|
|  ADBE|2018-02-28|
|  ADBE|2018-03-01|
|  ADBE|2018-03-02|
|  ADBE|2018-03-05|
|  ADBE|2018-03-06|
|  ADBE|2018-03-07|
|  ADBE|2018-03-08|
|  ADBE|2018-03-09|


In [8]:
# Filter stocks based on following criteria:
# - Minervini trend template
# - Computers related
# - Earnings > 20, Sales > 15
# Ordered by 21-day relative strength
q = spark.sql("""
WITH RSRATING AS (
    SELECT Ticker, Date, 
       Strength, 
       int(percent_rank(Strength) over (partition by Date order by Strength asc) * 100) RS
    FROM prices P
  ),
Q AS (
SELECT P.Ticker, P.Date, 
       --Open, Close, PrevClose, ATR,
       SMA50, SMA200, 
       Volume, Volume14MA,
       R.Strength, R.RS
FROM prices P, universe U, RSRating R 
where
     P.Ticker = U.Symbol and P.Ticker = R.Ticker and P.Date = R.Date
  and P.Date = '2021-10-04' 
  and (
      EG0 >= 20  --After 3Q 2020
  and SG0 >= 15

  and EG1 >= 20 
  and SG1 >= 15

  and close >= 25
  and close >= SMA50*0.95
  and sma50 >= sma200
  --and close >= 0.75 * HIGH52WK
  --and close >= 1.25 * LOW52WK
  --and Volume14MA > 250000
  and Volume14MA * SMA50 > 25e6)
  ORDER BY RS Desc 
)
SELECT count(*) from Q

--((Close - Open) >= ATR or (Close - PrevClose) >= ATR)
--and Volume >= Volume14MA
""")
q.show(200)


#for ticker in q.toPandas().Ticker:
#    print(ticker)

+--------+
|count(1)|
+--------+
|     146|
+--------+



In [9]:
dfUniverse[dfUniverse.Symbol == 'ORLY'].to_dict(orient='list')

{'Order': [2272],
 'Symbol': ['ORLY'],
 'Name': ['O Reilly Automotive Inc'],
 'Industry_Name': ['Retail/Whlsle-Auto Parts'],
 'Ind_Group_Rank': [37],
 'Sector': ['RETAIL'],
 'Shares_1000s': [68951],
 'Shares_in_Float_1000s': [66882],
 'Market_Cap_mil': [41885.7],
 'Funds_Perc': [46.0],
 'Funds_Perc_Increase': [6.5],
 'Mgmt_Perc': [2.0],
 'Number_of_Funds': [1954.0],
 'Inst_Ownership_Lss_Median': [0.0],
 'Sponsor_Rating': ['B'],
 'Enterprise_Val_mil': [45326.8],
 'EPS_Rating': [92.0],
 'RS_Rating': [81.0],
 'SMR_Rating': ['A'],
 'AD_Rating': ['D'],
 'Comp_Rating': [92.0],
 'Ind_Group_RS': ['A-'],
 'EPS_Lst_Rptd': ['2021-07-28'],
 'EPS_Due_Date': ['2021-10-28'],
 'EPS_Perc_Chg_Last_Qtr': [17.0],
 'EG0': [17.0],
 'EPS_Perc_Chg_1_Q_Ago': [78.0],
 'EG1': [78.0],
 'EPS_Perc_Chg_2_Q_Ago': [27.0],
 'EG2': [27.0],
 'EPS_Perc_Chg_3_Q_Ago': [39.0],
 'EG3': [39.0],
 'EPS_Accel_3_Qtrs': [0],
 'EPS_Trailing_4_Qtrs': [27.86],
 'EPS_Perc_Growth_5_Yr_Pct_Rnk': [79.0],
 'EPS_Est_Next_Yr_Perc': [6.0],
 '

In [24]:
df = spark.sql("""
  WITH RSRATING AS (
    SELECT Ticker, Industry_name, Date, Close,
           Strength21, int(percent_rank(Strength21) over (partition by Date order by Strength21 asc) * 100) RS21,
           Strength42, int(percent_rank(Strength42) over (partition by Date order by Strength42 asc) * 100) RS42,
           Strength63, int(percent_rank(Strength63) over (partition by Date order by Strength63 asc) * 100) RS63,
           Strength, int(percent_rank(Strength) over (partition by Date order by Strength asc) * 100)     RS
    FROM prices P, universe U 
    where
         P.Ticker = U.Symbol)
  select * from RSRATING
    where 
      Date = '2020-04-06'
      and Industry_name like "Computer%"
      --and Ticker in ('ADBE')
    order by Date asc, RS21 desc
""").show(30)

#df
#df.RSRating.rolling(14).mean().plot()
#df.RSRating.tail(23)

+------+--------------------+----------+------+----------+----+----------+----+----------+----+--------+---+
|Ticker|       Industry_name|      Date| Close|Strength21|RS21|Strength42|RS42|Strength63|RS63|Strength| RS|
+------+--------------------+----------+------+----------+----+----------+----+----------+----+--------+---+
|    ZS|Computer Sftwr-Se...|2020-04-06| 63.31|     28.18|  99|     22.19|  99|     22.55|  99|    12.9| 95|
|  NTGR| Computer-Networking|2020-04-06|  22.6|     20.99|  99|     10.83|  98|      4.48|  96|  -20.24| 67|
|  AUDC|Computer Sftwr-En...|2020-04-06| 26.39|     21.11|  99|     19.43|  99|     16.96|  98|   38.72| 98|
|  CRWD|Computer Sftwr-Se...|2020-04-06| 59.07|     13.68|  98|      8.25|  97|      6.95|  97|    null|  0|
|  LOGI|Computer-Hardware...|2020-04-06| 45.02|     13.14|  98|      8.67|  97|      5.13|  96|    5.42| 91|
|  OSPN|Computer Sftwr-Se...|2020-04-06|  17.6|     15.49|  98|     11.43|  98|      7.99|  97|    8.04| 93|
|  OPRX|Computer Sf

In [11]:
[c for c in dfUniverse.columns if c.find("Vol") >= 0]
#list(dfUniverse.columns)
cols = ["Symbol", 'Industry_Name', 'Comp_Rating', 'EPS_Rating', 'Ind_Group_Rank', 'Ind_Group_RS', 'Annual_Sales_mil', "EG0", "EG1", "EG2", "EG3", "EG4", "EG5", "SG0", "SG1", "SG2", "SG3", "SG4", "SG5"]
dfUniverse[(dfUniverse.Symbol == 'TGT')][cols]

#dfUniverse[(dfUniverse.Symbol == 'STLD')].to_dict(orient='list')

['50_Day_Avg_Vol_1000s',
 '50_Day_Avg_Dollar_Vol_1000s',
 'Volume_1000s',
 'Dollar_Volume_mil',
 'Dollar_Volume_Sales']

,Symbol,Industry_Name,Comp_Rating,EPS_Rating,Ind_Group_Rank,Ind_Group_RS,Annual_Sales_mil,EG0,EG1,EG2,EG3,EG4,EG5,SG0,SG1,SG2,SG3,SG4,SG5
2962,TGT,Retail-Major Disc Chains,85.0,84.0,39,A-,93561.0,8.0,525.0,58.0,105.0,85.6,-61.4,10.0,22.8,21.2,21.2,24.8,11.0


In [13]:
dfUniverse[cond].Symbol[:50]

63        AEM
146      AMBA
151       AMD
182        AN
200      APAM
267      ATKR
294      AVNT
342      BBDC
350        BC
352       BCC
720      COIN
764       CRK
907      DIOD
936      DOCS
947        DQ
988      ECHO
1014     ELAN
1019      ELY
1026     ENPH
1101     EXPI
1110     FANG
1115       FB
1192      FND
1196      FNV
1316     GLOB
1340     GOOG
1341    GOOGL
1435     HIMX
1439      HLI
1581     INMD
1702     KLIC
1740      LAD
1787     LGND
1828      LPX
1838     LSTR
1859      LZB
1969     MNSO
1994     MPWR
2007     MRVI
2239     OLPX
2331      PCH
2362      PFE
2431     POWI
2476     PUBM
2482      PWP
2492     QCOM
2524     RCII
2544     REGN
2669     SCCO
2671     SCHN
Name: Symbol, dtype: object

In [59]:
dfUniverse[(dfUniverse.Symbol == 'ABCL')].to_dict(orient='list')


{'Order': [15],
 'Symbol': ['ABCL'],
 'Name': ['Abcellera Biologics Inc'],
 'Industry_Name': ['Medical-Biomed/Biotech'],
 'Ind_Group_Rank': [42],
 'Sector': ['MEDICAL'],
 'Shares_1000s': [278703],
 'Shares_in_Float_1000s': [153286],
 'Market_Cap_mil': [4914.9],
 'Funds_Perc': [31.0],
 'Funds_Perc_Increase': [26.7],
 'Mgmt_Perc': [44.0],
 'Number_of_Funds': [76.0],
 'Inst_Ownership_Lss_Median': [0.0],
 'Sponsor_Rating': ['B'],
 'Enterprise_Val_mil': [4792.6],
 'EPS_Rating': [29.0],
 'RS_Rating': [19.0],
 'SMR_Rating': ['B'],
 'AD_Rating': ['B'],
 'Comp_Rating': [39.0],
 'Ind_Group_RS': ['A-'],
 'EPS_Lst_Rptd': ['2021-08-12'],
 'EPS_Due_Date': ['2021-10-28'],
 'EPS_Perc_Chg_Last_Qtr': [nan],
 'EG0': [-133.0],
 'EPS_Perc_Chg_1_Q_Ago': [nan],
 'EG1': [3800.0],
 'EPS_Perc_Chg_2_Q_Ago': [nan],
 'EG2': [4500.0],
 'EPS_Perc_Chg_3_Q_Ago': [nan],
 'EG3': [-200.0],
 'EPS_Accel_3_Qtrs': [0],
 'EPS_Trailing_4_Qtrs': [0.79],
 'EPS_Perc_Growth_5_Yr_Pct_Rnk': [nan],
 'EPS_Est_Next_Yr_Perc': [nan],
 'E